In [ ]:
!pip install prophet --quiet

In [7]:
#prophet with train and test data
import pandas as pd
from prophet import Prophet
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score

#forecast settings
forecast_steps = 17280  #6 months of 15-min intervals
cell_col = "NCI"
timestamp_col = "Timestamp"
test_ratio = 0.2  #20% for testing

def forecast_with_prophet(df, kpi_col, output_path):
    all_forecasts = []
    accuracy_records = []

    for cell in df[cell_col].unique():
        print(f"\nForecasting for cell {cell} | KPI: {kpi_col}")
        cell_df = df[df[cell_col] == cell][[timestamp_col, kpi_col]].dropna()

        try:
            cell_df[timestamp_col] = pd.to_datetime(cell_df[timestamp_col], format="%m-%d-%Y %H:%M:%S", errors='coerce')
            cell_df = cell_df.dropna().sort_values(timestamp_col)
        except Exception as e:
            print(f"Failed to parse timestamps for cell {cell}: {e}")
            continue

        ts = cell_df.rename(columns={timestamp_col: "ds", kpi_col: "y"})

        #split into train and test sets
        split_idx = int(len(ts) * (1 - test_ratio))
        train_df = ts.iloc[:split_idx]
        test_df = ts.iloc[split_idx:]

        #fit model on training set
        model = Prophet(daily_seasonality=True, weekly_seasonality=True)
        model.fit(train_df)

        #predict on test set to evaluate accuracy
        forecast_test = model.predict(test_df[["ds"]])
        accuracy = r2_score(test_df["y"], forecast_test["yhat"]) * 100
        print(f"Test Accuracy for cell {cell}: {accuracy:.2f}%")

        accuracy_records.append({
            "Cell": cell,
            "KPI": kpi_col,
            "Accuracy(%)": round(accuracy, 2)
        })

        #forecast into the future
        future_start = pd.Timestamp("2025-05-02 00:00:00")
        future_end = future_start + pd.Timedelta(minutes=15 * forecast_steps)
        future = pd.date_range(start=future_start, end=future_end, freq="15min")
        future_df = pd.DataFrame({"ds": future})

        forecast_future = model.predict(future_df)
        forecast_future = forecast_future[forecast_future["ds"] > ts["ds"].max()]

        forecast_future["nCI"] = cell
        forecast_future["KPI"] = kpi_col
        forecast_future.rename(columns={"ds": "Timestamp", "yhat": "Forecast"}, inplace=True)
        all_forecasts.append(forecast_future[["Timestamp", "Forecast", "nCI", "KPI"]])

    #combine forecasts and save
    final_df = pd.concat(all_forecasts, ignore_index=True)
    final_df.to_csv(output_path, index=False)
    print(f"\nForecasts saved to: {output_path}")

    accuracy_df = pd.DataFrame(accuracy_records)
    accuracy_output_path = output_path.replace(".csv", "_accuracy.csv")
    accuracy_df.to_csv(accuracy_output_path, index=False)
    print(f"Accuracy report saved to: {accuracy_output_path}")

#load data
df_prb = pd.read_csv("/content/drive/MyDrive/DL_Prb_Utilization_Data.csv")
df_ue = pd.read_csv("/content/drive/MyDrive/Avg_UE_Number_Data.csv")

#run forecasting
forecast_with_prophet(df_prb, "DL_Prb_Utilization", "/content/drive/MyDrive/forecast_DL_Prb_Utilization_Prophet.csv")
forecast_with_prophet(df_ue, "Avg_UE_Number", "/content/drive/MyDrive/forecast_Avg_UE_Number_Prophet.csv")


Forecasting for cell 357783981 | KPI: DL_Prb_Utilization


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/cidskt6z.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/gij0x5ow.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=71093', 'data', 'file=/tmp/tmp0tfn0ytv/cidskt6z.json', 'init=/tmp/tmp0tfn0ytv/gij0x5ow.json', 'output', 'file=/tmp/tmp0tfn0ytv/prophet_modelcdnkd7dh/prophet_model-20250620181500.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:15:00 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:15:03 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Test Accuracy for cell 357783981: 79.65%

Forecasting for cell 357783979 | KPI: DL_Prb_Utilization


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/yw1rqm2b.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/gn947vlb.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=76929', 'data', 'file=/tmp/tmp0tfn0ytv/yw1rqm2b.json', 'init=/tmp/tmp0tfn0ytv/gn947vlb.json', 'output', 'file=/tmp/tmp0tfn0ytv/prophet_modelsnyz4jvg/prophet_model-20250620181511.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:15:11 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:15:17 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Test Accuracy for cell 357783979: 79.10%


...

Test Accuracy for cell 358531245: 70.41%


INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/ktgtikhx.json



Forecasting for cell 358531243 | KPI: Avg_UE_Number


DEBUG:cmdstanpy:input tempfile: /tmp/tmp0tfn0ytv/e7gb3fe4.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=28116', 'data', 'file=/tmp/tmp0tfn0ytv/ktgtikhx.json', 'init=/tmp/tmp0tfn0ytv/e7gb3fe4.json', 'output', 'file=/tmp/tmp0tfn0ytv/prophet_modelc20n_yp5/prophet_model-20250620181710.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
18:17:10 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
18:17:14 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


Test Accuracy for cell 358531243: 70.88%

Forecasts saved to: /content/drive/MyDrive/forecast_Avg_UE_Number_Prophet.csv
Accuracy report saved to: /content/drive/MyDrive/forecast_Avg_UE_Number_Prophet_accuracy.csv
